In [1]:
import os
import pandas as pd
import duckdb
from dotenv import load_dotenv
# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Acessar as variáveis de ambiente
pasta_csv = os.getenv('PASTA_CSV')
banco_dados_duckdb = os.getenv('BANCO_DADOS_DUCKDB')
pasta_parquet = os.getenv('PASTA_PARQUET')

def connetionFactory():
    try:
        # Conexão com o banco de dados DuckDB
        conn = duckdb.connect(banco_dados_duckdb)
        return conn
    except Exception as e:
        print(f" ERROR <=> {e} ... ")    
        return None
    

def main():
    print(f"  >> INICIANDO PROCESSO DE CRIAÇÃO DE BASES F1")
    try:
        conn = connetionFactory()
        if conn:
            # Iterando sobre os arquivos na pasta
            for arquivo in os.listdir(pasta_csv):
                if arquivo.endswith('.csv'):
                    nome_arquivo = os.path.splitext(arquivo)[0]
                    print(f"    ** Lendo Arquivo => {arquivo} <= ")
                    caminho_csv = os.path.join(pasta_csv, arquivo)
                    
                    # Ler o CSV
                    df = pd.read_csv(caminho_csv)
                    
                    # Criar o Parquet
                    caminho_parquet = os.path.join(pasta_parquet, f'{nome_arquivo}.parquet')
                    print(f"    ** Criando Arquivo .Parquet ")
                    df.to_parquet(caminho_parquet, index=False)
                    
                    # Verificar se a tabela existe
                    check_table_query = f"""
                    SELECT COUNT(*) FROM information_schema.tables 
                    WHERE table_name = '{nome_arquivo}';
                    """
                    result = conn.execute(check_table_query).fetchone()[0]

                    # Se a tabela existe, remover
                    if result > 0:
                        print(f"    ** Tabela {nome_arquivo} existe. Removendo...")
                        conn.execute(f"DROP TABLE {nome_arquivo};")

                    # Criar a nova tabela no DuckDB a partir do Parquet
                    conn.execute(f"""
                    CREATE TABLE {nome_arquivo} AS
                    SELECT * FROM read_parquet('{caminho_parquet}');
                    """)
                    print(f"    ** Tabela {nome_arquivo} criada no duckdb")
                    print(f"    *****************************************")
                    print("")

        print(f"  >> FINALIZANDO PROCESSO DE CRIAÇÃO DE BASES F1")            
    except Exception as e:
        print(f"   >> ERROR NO PROCESSO <=> {e}")
    finally:
        if conn:
            conn.close()
            print("Conexão com o DuckDB fechada.")


if __name__ == "__main__":
    main()


  >> INICIANDO PROCESSO DE CRIAÇÃO DE BASES F1
    ** Lendo Arquivo => circuits.csv <= 
    ** Criando Arquivo .Parquet 
    ** Tabela circuits existe. Removendo...
    ** Tabela circuits criada no duckdb
    *****************************************

    ** Lendo Arquivo => status.csv <= 
    ** Criando Arquivo .Parquet 
    ** Tabela status existe. Removendo...
    ** Tabela status criada no duckdb
    *****************************************

    ** Lendo Arquivo => lap_times.csv <= 
    ** Criando Arquivo .Parquet 
    ** Tabela lap_times existe. Removendo...
    ** Tabela lap_times criada no duckdb
    *****************************************

    ** Lendo Arquivo => sprint_results.csv <= 
    ** Criando Arquivo .Parquet 
    ** Tabela sprint_results existe. Removendo...
    ** Tabela sprint_results criada no duckdb
    *****************************************

    ** Lendo Arquivo => drivers.csv <= 
    ** Criando Arquivo .Parquet 
    ** Tabela drivers existe. Removendo...
  

In [38]:
conn = connetionFactory()
df = conn.execute("SELECT * FROM races  where year = 2024").fetchdf()
display(df)


,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1121,2024,1,3,Bahrain Grand Prix,2024-03-02,15:00:00,https://en.wikipedia.org/wiki/2024_Bahrain_Gra...,2024-02-29,11:30:00,2024-02-29,15:00:00,2024-03-01,12:30:00,2024-03-01,16:00:00,\N,\N
1,1122,2024,2,77,Saudi Arabian Grand Prix,2024-03-09,17:00:00,https://en.wikipedia.org/wiki/2024_Saudi_Arabi...,2024-03-07,13:30:00,2024-03-07,17:00:00,2024-03-08,13:30:00,2024-03-08,17:00:00,\N,\N
2,1123,2024,3,1,Australian Grand Prix,2024-03-24,04:00:00,https://en.wikipedia.org/wiki/2024_Australian_...,2024-03-22,01:30:00,2024-03-22,05:00:00,2024-03-23,01:30:00,2024-03-23,05:00:00,\N,\N
3,1124,2024,4,22,Japanese Grand Prix,2024-04-07,05:00:00,https://en.wikipedia.org/wiki/2024_Japanese_Gr...,2024-04-05,02:30:00,2024-04-05,06:00:00,2024-04-06,02:30:00,2024-04-06,06:00:00,\N,\N
4,1125,2024,5,17,Chinese Grand Prix,2024-04-21,07:00:00,https://en.wikipedia.org/wiki/2024_Chinese_Gra...,2024-04-19,03:30:00,2024-04-19,07:30:00,\N,\N,2024-04-20,07:00:00,2024-04-20,03:00:00
5,1126,2024,6,79,Miami Grand Prix,2024-05-05,20:00:00,https://en.wikipedia.org/wiki/2024_Miami_Grand...,2024-05-03,16:30:00,2024-05-03,20:30:00,\N,\N,2024-05-04,20:00:00,2024-05-04,16:00:00
6,1127,2024,7,21,Emilia Romagna Grand Prix,2024-05-19,13:00:00,https://en.wikipedia.org/wiki/2024_Emilia_Roma...,2024-05-17,11:30:00,2024-05-17,15:00:00,2024-05-18,10:30:00,2024-05-18,14:00:00,\N,\N
7,1128,2024,8,6,Monaco Grand Prix,2024-05-26,13:00:00,https://en.wikipedia.org/wiki/2024_Monaco_Gran...,2024-05-24,11:30:00,2024-05-24,15:00:00,2024-05-25,10:30:00,2024-05-25,14:00:00,\N,\N
8,1129,2024,9,7,Canadian Grand Prix,2024-06-09,18:00:00,https://en.wikipedia.org/wiki/2024_Canadian_Gr...,2024-06-07,17:30:00,2024-06-07,21:00:00,2024-06-08,16:30:00,2024-06-08,20:00:00,\N,\N
9,1130,2024,10,4,Spanish Grand Prix,2024-06-23,13:00:00,https://en.wikipedia.org/wiki/2024_Spanish_Gra...,2024-06-21,11:30:00,2024-06-21,15:00:00,2024-06-22,10:30:00,2024-06-22,14:00:00,\N,\N
